In [1]:
import h5py
import glob
from tables import *
import numpy as np
import pandas as pd
import ROOT
from ROOT import gROOT 
from ROOT import gStyle
%jsroot on


Welcome to JupyROOT 6.26/10


In [2]:
ROOT.EnableImplicitMT()
gStyle.SetOptStat("0000000")

In [3]:
def processfiles(h5_files, nmax=800):
    
    print(len(h5_files))
    h5file = open_file(h5_files[0], mode="r")
    group = h5file.get_node("/dl2/eventlist")
    group
    result=[]
    for n, h5file in enumerate(h5_files):
        df1 = pd.read_hdf(h5file, key="/dl2/eventlist")
        if n == 0:
            frames = [df1]
        else:
            frames = [df1, result]
        result = pd.concat(frames)
        #print(n, h5file)
        if n>nmax:
            break
        
    print(len(result))
    print(n)
    result=result.sort_values("tstart", ascending=True).reset_index(drop=True)
    #extract data
    halflife=result['halflife'].to_numpy()
    temp = result['temp'].to_numpy()
    peak=result['peak'].to_numpy()
    tstart=result['tstart'].to_numpy()
    index_peak=result['index_peak'].to_numpy()
    mult=result['mult'].to_numpy()
    n_waveform=result['n_waveform'].to_numpy()
    integral1=result['integral1'].to_numpy()
    integral2=result['integral2'].to_numpy()
    integral3=result['integral3'].to_numpy()
    
    print('nsecs ' + str(result.tstart[len(tstart)-1] - result.tstart[0]))

    df = ROOT.RDF.MakeNumpyDataFrame({'tstart': tstart, 'halflife': halflife, 'index_peak': index_peak, 'n_waveform': n_waveform,  'temp': temp, 'peak': peak, 'mult': mult, 'integral1':integral1, 'integral2':integral2, 'integral3':integral3})
    df.Describe()

    return df

In [4]:


#print("DL2/RPG104/25mV")
#h5_files = sorted(glob.glob("/data02/gammaflash/PL/DL2/RPG104/25mV/*.dl2.h5"))
#df1 = processfiles(h5_files)
#df1txt="BKG/RPG104/25mV"

#print("DL2/RPG104/15mV")
#h5_files = sorted(glob.glob("/data02/gammaflash/PL/DL2/RPG104/15mV/*.dl2.h5"))
#df1 = processfiles(h5_files)
#df1txt="BKG/RPG104/15mV"

print("DL2/RPG104/100mV")
h5_files = sorted(glob.glob("/data02/gammaflash/PL/DL2/RPG104/100mV/*.dl2.h5"))
df1 = processfiles(h5_files)
df1txt="BKG/RPG104/100mV"


DL2/RPG104/100mV
7701
1169273
801
nsecs 18781.25851416588


# Technical parameters

In [5]:
df = df1

In [6]:
c = ROOT.TCanvas("halflife", "halflife", 1024, 768)
hist1=df.Histo1D(("halflife", "halflife", 100, 0, 200), "halflife")
hist1.Draw()
c.Draw()

In [7]:
c = ROOT.TCanvas("index_peak", "index_peak", 1024, 768)
hist1=df.Histo1D(("index_peak", "index_peak", 180, 0, 18000), "index_peak")
c.SetLogy()
hist1.Draw()
c.Draw()

In [8]:
c = ROOT.TCanvas("mult", "mult", 1024, 768)
hist1=df.Histo1D(("mult", "mult", 20, 0, 20), "mult")
c.SetLogy()
hist1.Draw()
c.Draw()

In [9]:
c = ROOT.TCanvas("n_waveform", "n_waveform", 1024, 768)
hist1=df.Histo1D(("n_waveform", "n_waveform", 20, 0, 20), "mult")
c.SetLogy()
hist1.Draw()
c.Draw()

# Spectra

In [10]:







histA=df1.Histo1D((df1txt, df1txt, 900, 0, 9000), "peak")
histA.Scale(1/histA.Integral())
histA.SetLineWidth(2)
histA.SetLineColor(1)



c1 = ROOT.TCanvas("peak", "peak", 1024, 768)
c1.SetLogx()
c1.SetLogy()

histA.Draw("HIST")


c1.BuildLegend()
c1.Draw()




In [11]:
df=df1
txt = df1txt

range1=df.Max("integral1").GetValue()
binsenergy=int(range1/100)

c1 = ROOT.TCanvas("integralSRC", "integralSRC", 1024, 768)
c1.SetLogx()
c1.SetLogy()

histA=df.Histo1D(("integral1-"+txt, "integral1-"+txt, binsenergy, 0, range1), "integral1")
histA.SetLineColor(1);
histA.SetLineWidth(2);
histA.Draw()

histB=df.Histo1D(("integral2-"+txt, "integral2-"+txt, binsenergy, 0, range1), "integral2")
histB.SetLineColor(2);
histB.Draw("SAME")

histC=df.Histo1D(("integral3-"+txt, "integral3-"+txt, binsenergy, 0, range1), "integral3")
histC.SetLineColor(3);
#histC.Draw("SAME")

c1.BuildLegend()
c1.Draw()

In [12]:
## CONFRONTO SRC/BKG




param="integral1"

range1=df1.Max(param).GetValue()
binsenergy=int(range1/100)

c1 = ROOT.TCanvas("integralSRC-BKG", "integralSRC-BKG", 1024, 768)
c1.SetLogx()
c1.SetLogy()


hist1=df1.Histo1D((param+"-"+df1txt, param+"-"+df1txt, binsenergy, 0, range1), param)
hist1.Scale(1/hist1.Integral() * 1)
hist1.SetLineColor(1)
hist1.SetLineWidth(2)
hist1.GetYaxis().SetRangeUser(0, 1)





hist1.Draw("HIST")
hist1.SaveAs("hit.root")

c1.BuildLegend()
c1.Draw()
c1.SaveAs("hist1.root")

Info in <TH1D::SaveAs>: ROOT file hit.root has been created
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Info in <TCanvas::SaveAs>: ROOT file hist1.root has been created


In [13]:
# Creazione di un oggetto TSpectrum per trovare i picchi
spectrum = ROOT.TSpectrum()

# Imposta i limiti del range in cui cercare i picchi
lower_limit = 100  # Sostituisci con il tuo limite inferiore
upper_limit = 700  # Sostituisci con il tuo limite superiore
numPeaks = spectrum.Search(hist1, 2, "", 0.05)

cp = ROOT.TCanvas("integralSRC-BKG", "integralSRC-BKG", 1024, 768)
cp.SetLogx()
cp.SetLogy()
hist.Draw()

for i in range(numPeaks):
    x_peak = spectrum.GetPositionX()[i]
    line = ROOT.TLine(x_peak, 0, x_peak, hist.GetMaximum())
    line.SetLineColor(ROOT.kRed)
    line.Draw("same")

c1.BuildLegend()
c1.Draw()

TypeError: int TSpectrum::Search(const TH1* hist, double sigma = 2, const char* option = "", double threshold = 0.050000000000000003) =>
    TypeError: could not convert argument 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Definizione della funzione Power Law
def power_law(x, A, n):
    return A * (x ** n)

# Dati di esempio
x_data = []
y_data = []

hist = hist1

for bin_idx in range(100, 700 + 1):
    x_bin_center = hist.GetBinCenter(bin_idx)
    y_bin_content = hist.GetBinContent(bin_idx)
    x_data.append(x_bin_center)
    y_data.append(y_bin_content)

if False:
    # Fit della curva
    params, covariance = curve_fit(power_law, x_data, y_data, p0=[1.0, -2.0])
    
    # Estrai i parametri fittati
    A, n = params
    
    # Calcola gli errori sui parametri dalla matrice di covarianza
    err_A, err_n = np.sqrt(np.diag(covariance))
    
    # Stampa i risultati
    print(f"Parametro A: {A} +/- {err_A}")
    print(f"Parametro n: {n} +/- {err_n}")

if True:
    # Grado del polinomio da adattare
    degree = 3  # Cambia il grado del polinomio a seconda delle tue esigenze
    
    # Fit del polinomio
    coefficients = np.polyfit(x_data, y_data, degree)
    
    # Costruisci il polinomio risultante
    polynomial = np.poly1d(coefficients)
    
    # Stampa i coefficienti del polinomio risultante
    print("Coefficienti del polinomio:")
    for i, coeff in enumerate(reversed(coefficients)):
        print(f"Coefficiente di grado {i}: {coeff}")

# Crea un grafico dei dati e della curva fittata in scala log-log
plt.figure(figsize=(8, 6))
plt.loglog(x_data, y_data, 'bo', label='Dati di esempio')  # 'bo' indica i punti blu
#plt.loglog(x_data, power_law(x_data, A, n), 'r-', label=f'Fit: A={A:.3f}, n={n:.3f}', color='red')
plt.loglog(x_data, polynomial(x_data), 'r-', label=f'Fit: Grado {degree}', linewidth=2)

plt.xlabel('X')
plt.ylabel('Y')
plt.legend()

# Salva il grafico in un file
plt.savefig("power_law_fit.png")

# Mostra il grafico
plt.show()


In [ ]:
histogram = hist1
with open("valori_dei_bin.txt", "w") as file_txt:
    for bin_index in range(1, histogram.GetNbinsX() + 1):
        bin_value = histogram.GetBinContent(bin_index)
        file_txt.write(f"Bin {bin_index}: {bin_value}\n")

In [ ]:
hist = hist1
hist.SetLineColor(1)
c3 = ROOT.TCanvas("integralSRC-BKG", "integralSRC-BKG", 1024, 768)
c3.SetLogx()
c3.SetLogy()

# Definire una funzione TF1 per la power law
def power_law(x, params):
    return params[0] * (x[0]**params[1])

power_law_func = ROOT.TF1("power_law_func", power_law, 10000, 66000, 2)  # 2 parametri

# Imposta stime iniziali per i parametri della power law
power_law_func.SetParameters(1000000, -2)

if False:
    # Creare una funzione TF1 per il fit dei picchi gaussiani
    num_peaks = 3  # Imposta il numero di picchi desiderati
    gaussian_peak_funcs = []
    
    for i in range(num_peaks):
        gaussian_peak_func = ROOT.TF1(f"gaussian_peak_func_{i}", "gaus", 0, 100)
        gaussian_peak_func.SetParameters(100, 20 * (i + 1), 5)  # Stime iniziali dei parametri
        gaussian_peak_funcs.append(gaussian_peak_func)
    
    # Combinare la power law e i picchi gaussiani
    total_fit_func = ROOT.TF1("total_fit_func", "[0]*power_law_func + [1]*gaussian_peak_func_0 + [2]*gaussian_peak_func_1 + [3]*gaussian_peak_func_2", 10000, 66000)
    total_fit_func.SetParameters(1, 1, 1, 1)  # Stime iniziali per i coefficienti

fit_function = power_law_func

print(fit_function(10000))

In [ ]:
# Eseguire il fit
#hist.Fit(power_law_func, "RV")

# Estrai i parametri fittati
A = fit_function.GetParameter(0)
n = fit_function.GetParameter(1)

# Calcola gli errori sui parametri
err_A = fit_function.GetParError(0)
err_n = fit_function.GetParError(1)

# Stampa i risultati
print(f"Parametro A: {A} +/- {err_A}")
print(f"Parametro n: {n} +/- {err_n}")

histogram.Draw("HIST")
fit_function.Draw("SAME")
c3.BuildLegend()
c3.Draw()

In [ ]:
c3.SaveAs("b.root")
c3.Close()

In [ ]:
df1=dfRPG101src25mV
txt = "dfRPG101src25mV"

c2 = ROOT.TCanvas("integralB", "integralB", 1024, 768)
c2.SetLogx()
c2.SetLogy()

histA.Scale(1/histA.Integral())
histA.Draw("HIST")

histB.Scale(1/histB.Integral())
histB.Draw("HISTSAME")

histA1=df1.Histo1D(("integral1-"+txt, "integral1-"+txt, binsenergy, 0, range1), "integral1")
histA1.SetLineColor(3);
histA1.SetLineWidth(2);
histA1.Scale(1/histA1.Integral())
histA1.Draw("HISTSAME")

histB1=df1.Histo1D(("integral2-"+txt, "integral2-"+txt, binsenergy, 0, range1), "integral2")
histB1.SetLineColor(4);
histB1.Scale(1/histB1.Integral())
histB1.Draw("HISTSAME")

c2.BuildLegend()
c2.Draw()

# Light curve

In [ ]:
tstart1 = tstart[0]
tend1 = tstart[len(tstart)-1]

binsizesecs = 10
df0 = df.Filter("tstart >= "+str(tstart1))

tmin=tstart1
tmax=tend1

nbins = (int(tmax) - int(tmin) + 1) / binsizesecs

print(tmin)
print(tmax)
print(tend1)
print(tmax-tmin)
print(int(nbins))

In [ ]:
lcRPG0 = df0.Histo1D(("time2", "RPG0 - "+str(binsizesecs)+" s - "+str(int(tmin))+"-"+str(int(tmax))+"", int(nbins), tmin, tmax), "tstart")
c = ROOT.TCanvas("LC1", "LC1", 1024, 768)

lcRPG0.Draw()
c.Draw()

# Temperature analysis dependencies

In [ ]:
stop

In [ ]:
df1 = df.Filter("temp < 10")
df2 = df.Filter("temp >= 10 and temp <15")
df3 = df.Filter("temp >= 15 and temp <20")
df4 = df.Filter("temp >= 25 and temp <30")
df5 = df.Filter("temp >= 30")

In [ ]:
c = ROOT.TCanvas("LC1", "LC1", 1024, 768)

hist1=df1.Histo1D(("halflife10", "halflife10", 100, 0, 200), "halflife")
hist1.Scale(1/hist1.Integral())

hist2=df2.Histo1D(("halflife15", "halflife15", 100, 0, 200), "halflife")
hist2.Scale(1/hist2.Integral())

hist3=df3.Histo1D(("halflife20", "halflife20", 100, 0, 200), "halflife")
hist3.Scale(1/hist3.Integral())

hist4=df4.Histo1D(("halflife25", "halflife25", 100, 0, 200), "halflife")
hist4.Scale(1/hist4.Integral())

hist5=df5.Histo1D(("halflife30", "halflife30", 100, 0, 200), "halflife")

hist2.SetLineColor(1);
hist1.SetLineColor(2);
hist3.SetLineColor(3);
hist4.SetLineColor(4);

hist2.Draw("HIST")
hist1.Draw("SAME")
hist3.Draw("SAME")
hist4.Draw("SAME")
c.BuildLegend()
c.Draw()

In [ ]:
hist2.SetLineColor(1);
hist1.SetLineColor(2);
hist3.SetLineColor(3);
hist4.SetLineColor(4);
hist4.Draw("HIST")
hist2.Draw("HISTSAME")
hist1.Draw("HISTSAME")
hist3.Draw("HISTSAME")

c.BuildLegend()
c.Draw()

# Peak analysis

In [ ]:
dfm1 = df.Filter("peak <= 1000")
dfm2 = df.Filter("peak > 1000 && peak < 8100")
dfm3 = df.Filter("peak > 8100")

In [ ]:
c2 = ROOT.TCanvas("mult", "mult", 1024, 768)
histm1=dfm1.Histo1D(("halflifem1", "halflifem1", 100, 0, 200), "halflife")
histm1.Scale(1/histm1.Integral())
histm2=dfm2.Histo1D(("halflifem2", "halflifem2", 100, 0, 200), "halflife")
histm2.Scale(1/histm2.Integral())
histm3=dfm3.Histo1D(("halflifem3", "halflifem3", 100, 0, 200), "halflife")
histm3.Scale(1/histm3.Integral())

histm1.SetLineColor(1);
histm2.SetLineColor(2);
histm3.SetLineColor(3);

histm1.Draw("HIST")
histm2.Draw("HISTSAME")
histm3.Draw("HISTSAME")

c2.BuildLegend()
c2.SetLogy()
c2.Draw()

In [ ]:
cint1 = ROOT.TCanvas("integral1", "integral1", 1024, 768)
histint1=df.Histo1D(("integral1", "integral1", 1000, 0, 2000000), "integral1")
histint2=df.Histo1D(("integral2", "integral2", 1000, 0, 2000000), "integral2")
histint3=df.Histo1D(("integral3", "integral3", 1000, 0, 2000000), "integral3")
cint1.SetLogx()
cint1.SetLogy()
histint1.SetLineColor(1);
histint2.SetLineColor(2);
histint3.SetLineColor(3);
histint1.Draw()

histint2.Draw("SAME")
histint3.Draw("SAME")
cint1.Draw()